In [17]:
import pandas as pd
import numpy as np

# load files
listings = pd.read_csv('data/listings.csv')
neighbourhoods = pd.read_csv('data/neighbourhoods.csv')
calendar = pd.read_csv('data/calendar.csv')
calendar.head()

,listing_id,date,available,price
0,20872145,2018-09-22,f,NaN
1,20872145,2018-09-21,f,NaN
2,20872145,2018-09-20,f,NaN
3,20872145,2018-09-19,f,NaN
4,20872145,2018-09-18,f,NaN


## Clean data in calendar.csv

In [29]:
df_calendar = calendar
df_listings = listings
df2 = df_calendar[df_calendar.available.str[0:] != "f"]

# create panda series for "sum_of_nights" indicating the number of available nights
a = df2.groupby('listing_id').listing_id.count()
df_nights = pd.DataFrame({'id':a.index, 'sum_of_nights':a.values})

In [43]:
# Create month column
df_calendar['date'] = pd.to_datetime(df_calendar['date'], errors='coerce')
df_calendar['month'] = df_calendar['date'].dt.month

# Adjust data type of price to float
dfa = df_calendar.fillna('$0.00')
dfa['price'] = dfa['price'].str.replace('$', '')
dfa['price'] = dfa['price'].str.replace(',', '')
dfa['price'] = dfa['price'].astype('float64')

In [44]:
# Calculate the mean price for each month
clear_nums = {'t':1, 'f':0}
dfa.replace(clear_nums, inplace = True)
mul = []
for i, row in dfa.iterrows():
    mul.append(row[2] * row[3])

In [51]:
# Calculate the mean price for each month and replace NAN with 0.
dfa['multiple'] = mul
summ = dfa.groupby(['listing_id', 'month']).multiple.sum()
ava_count = dfa.groupby(['listing_id', 'month']).available.sum()
monthly_price = summ/ava_count
dfb = monthly_price.reset_index().fillna(0)

In [49]:
# helper function for changing the format of calendar table
lists_id = []
for i in dfb['listing_id']:
    if i not in lists_id:
        lists_id.append(i)
dfc = pd.DataFrame(lists_id, columns = ['id'])
dfc.head()

,id
0,3781
1,5506
2,6695
3,6976
4,8792


In [52]:
#  changing the format of calendar table
def recursive(df, flag):
    price_list = []
    if flag == 13:
        return df
    else:
        for i, row in dfb.iterrows():
            if int(row[1]) == flag:
                price_list.append(row[2])
        df[str(flag)] = price_list
        flag += 1
        return recursive(df, flag) 
df_month = recursive(dfc, 1) 


In [8]:
# Save to csv files
df_month.to_csv('data/cleansed_calendar.csv', index=False)

In [53]:
# merge two dataframes, based on id, add the new column to 'listings'
df5 = pd.merge(df_listings, df_nights, on='id', how='left')
df7 = df5.fillna(0)

# check if there is a null value in id column
df7.id[df7.id == 0].count()


0

In [10]:
# Delete unimportant features
df8 = df7[df7.columns[[0,1,4,5,6,7,9,17,25,26,37,39,41,48,49,
                       50,51,52,53,54,55,56,58,60,89,91,96]]]
# check if there is any property with 'price == 0'
df8.price[df8.price == float(0)].count()

0

In [11]:
# Delete rows which 'beds == 0'
df9 = df8[df8.beds != 0]

In [12]:
# In column 'price', delete dollar sign($) and comma(,), convert datatype from string to float
df9['price'] = df9['price'].str.replace('$', '')
df9['price'] = df9['price'].str.replace(',', '')
df9['price'] = df9['price'].astype('float64') 


/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [13]:
# catagorize amenites
heating = []
wifi = []
tv = []
pets = []
Washer_dryer = []
gym = []

def split_amenities(each_loop, string, l):
    if string in each_loop:
        l.append(1)
    else:
        l.append(0)
        
def split(df9):
    for i in df9['amenities']:
        split_amenities(i, 'Heating', heating)
        split_amenities(i, '"Wireless Internet"', wifi)
        split_amenities(i, 'TV', tv)
        split_amenities(i, '"Pets allowed"', pets)
        split_amenities(i, 'Washer,Dryer', Washer_dryer)
        split_amenities(i, 'Gym', gym)

def sum_amenities(df):
    length = []
    for i in df['amenities']:
        s = i.replace("{", "").replace("}", "").split(",")
        length.append(len(s))
    return length
sum_of_amenities = sum_amenities(df9)
split(df9)


In [14]:
df9['heating'] = heating
df9['wifi'] = wifi
df9['pets_allowed'] = pets
df9['Washer_dryer'] = Washer_dryer
df9['gym'] = gym
df9['sum_of_amenities'] = sum_of_amenities

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pand

In [15]:
df9 = df9[['id', 
'listing_url', 
'name', 
'summary', 
'space', 
'description', 
'neighborhood_overview', 
'picture_url', 
'host_response_time', 
'host_response_rate', 
'street', 
'neighbourhood_cleansed', 
'city', 
'latitude', 
'longitude', 
'is_location_exact', 
'property_type', 
'room_type', 
'accommodates', 
'bathrooms', 
'bedrooms', 
'beds', 
'amenities', 
'heating', 
'wifi', 
'pets_allowed', 
'Washer_dryer', 
'gym', 
'sum_of_amenities', 
'price', 
'instant_bookable', 
'cancellation_policy', 
'sum_of_nights']]

In [16]:
# Save to csv files
df9.to_csv('data/cleansed_listings.csv', index=False)